In [32]:
import pandas as pd
import glob
from datetime import datetime
import matplotlib.pyplot as plt

In [42]:
# helper functions for log parsing

def pass_no_pass(lines):
    for line in lines:
        if "PASSED" in line:
            return True
        elif "FAILED" in line:
            return False
    return

def parse_fp_for_int_diff(lines):
    vals = {'expected': [], 'actual': []}
    delimiters = [" ", "\n"]
    for i in range(len(lines)):
        if "INT" in lines[i]:
            for delimiter in delimiters:
                line = " ".join(lines[i+1].split(delimiter))
            split = list(map(int, line.split()))
            if ("Expected value of" in lines[i]) and ("in" not in lines[i]):
                # print(lines[i])
                vals['expected'].append(split)
            elif "Last value of" in lines[i]:
                # print(lines[i])
                vals['actual'].append(split)

    diffs = []
    for i in range(len(vals['actual'])):
        for j in range(len(vals['actual'][i])):
            if (vals['expected'][i][j]+1) != vals['actual'][i][j]:
                diffs.append(vals['expected'][i][j]+1 - vals['actual'][i][j])
    return diffs


In [43]:
# create csv of testing logs
# change bool to generate csv if not created
gen_csv = True

def generate_csv():

    df = pd.DataFrame(columns=["kind", "run", "log", "lines", "time", "name", "path", "iters", "expected_vs_actual", "diff_count", "passed"])

    for filename in glob.iglob('**', recursive=True):
        # skip exceptions, keep .txts with logs
        if "log" not in filename or filename[-4:] != ".txt" or "exception" in filename:
            continue
        tmp = filename.split('/')
        if not len(tmp) - 1:
            continue
        fp = open(filename, 'r')
        lines = fp.readlines()
        fp.close()

        kind, name = tmp[0], tmp[-1]

        # TODO: something with GXL logs
        if kind == "gxld":
            continue

        # get iteration count
        iter = [line for line in lines if "Exited" in line][0].split()[-1]
        # get count of i/o matches
        int_diff = parse_fp_for_int_diff(lines)
        # get pass/fail
        passed = pass_no_pass(lines)


        # parse datetime from file name
        try:
            time = datetime.strptime(filename[-21:-4], '%Y%m%d_%H-%M-%S') # everything but GXLD logs
        except Exception:
            try:
                time = datetime.strptime(filename[-18:-4], '%m-%d-%y_%H-%M') # specifically for GXLD logs
            except Exception as e:
                raise(e)
        
        # get 'run' and 'log' numbers
        runlog = name.split('_')[2]
        run, log = [int(x) for x in runlog.split('run')[-1].split('log')]

        row = {"kind": kind, "run":run, "log": log, "lines": len(lines), "iters": int(iter), "time": time, "name": name,\
            "path": filename, "expected_vs_actual": int_diff, "diff_count": len(int_diff), "passed": passed}
        if df.name.str.contains(name).any():
            continue
        else:
            df.loc[len(df), :] = row
    df = df.sort_values(by=['kind', 'run', 'log']).reset_index(drop=True)
    df.to_csv("./results.csv", index=False)
if gen_csv:
    generate_csv()
df = pd.read_csv("./results.csv")

In [44]:
df

,kind,run,log,lines,time,name,path,iters,expected_vs_actual,diff_count,passed
0,mmb_static,1,1,30,2024-02-14 15:51:09,log_staticMMBtest_run1log1_20240214_15-51-09.txt,mmb_static/log_staticMMBtest_run1log1_20240214...,1,[65536],1,False


In [50]:
vs = df.expected_vs_actual[0][1:-1].split(',')
vs = list(map(int, vs))
sum([x == -1 for x in vs]) * 2

0

In [45]:
masks = []
for path in df.path:
    lines = open(path).read().split('\n')
    mask, mask_bool = [], False
    for line in lines:
        if "mask" in line:
            mask_bool = True
            continue
        elif ("SECTION" in line) or (line == ""):
            mask_bool = False
        if mask_bool:
            mask.append(line)
    if len(mask): masks.append(mask)

In [51]:
len(masks[0][0])*2

65536